In [1]:
from pytrends.request import TrendReq
import pandas as pd
import re
import matplotlib.pyplot as plt
import scipy.stats as sci
import numpy as np

df = pd.read_csv('symptoms.csv', sep=',',encoding='latin-1')


In [71]:
#Test call to see whether I am still blocked by Google,lol
test = 'Husten'
pytrend.build_payload(kw_list=['{}'.format(test)],geo="DE",timeframe='today 5-y',cat=0)
google_trend_per_syndrom = pytrend.interest_over_time()
google_values_unnormlised = google_trend_per_syndrom['{}'.format(test)].values
google_values= google_values_unnormlised/max(google_values_unnormlised)+0.0000001
print(google_values)


In [4]:
#If there are words in brackets and they start with a capital letter, keep and discard the rest
def word_in_bracket(x):
    match = re.search(r'.+?\(([A-Z].+)\)',x)
    if match is not None:
        return match.group(1)
    else:
        return x
    

In [5]:
#Cleaning of the data, as in the preprocessings scripts
df = pd.read_csv('symptoms.csv', sep=',',encoding='latin-1')
df.reset_index(drop=True)
df = df.drop(columns=['Unnamed: 0','[PathogenMitKrankheit].[Meldeweg Web71].[Meldeweg ID Web71].[MEMBER_CAPTION]'])
df = df.rename(columns={'[PathogenMitKrankheit].[Meldeweg Web71].[Krankheit ID Web71].[MEMBER_CAPTION]': 'Krankheit', '[Symptome].[ID].[ID].[MEMBER_CAPTION]': 'Symptom', '[ReportingDate].[YearWeek].[YearWeek].[MEMBER_CAPTION]': 'Kalenderwoche', '[Measures].[FallCount]': 'Anzahl'}) 


df_selection = df.drop(columns='Krankheit')
df_selection['Symptom'] = df_selection['Symptom'].apply(lambda x: 'Durchfall' if x =="Durchfall und/oder Erbrechen" else x)
df_selection['Symptom'] = df_selection['Symptom'].apply(lambda x: 'Durchfall' if x =="Durchfall, nicht näher bezeichnet" else x)
df_selection['Symptom'] = df_selection['Symptom'].apply(lambda x: 'Muskel-, Glieder-, Rücken- oder Kopfschmerzen' if x =="Gliederschmerzen" else x)
df_selection['Symptom'] = df_selection['Symptom'].apply(lambda x: 'Hirnhautentzündung' if x.startswith("Meningitis") else x)
df_selection['Symptom'] = df_selection['Symptom'].apply(word_in_bracket)


df_selection_pivot = df_selection.groupby(['Symptom','Kalenderwoche'])['Anzahl'].sum().reset_index()
df_selection_pivot.sort_values('Kalenderwoche')
df_selection_all_symptoms = df_selection.groupby(['Symptom'])['Anzahl'].sum().reset_index()['Symptom'].tolist()



new_rows = []
for week in df['Kalenderwoche'].drop_duplicates().sort_values():
    for symptom in df_selection_all_symptoms:
        sample_week = df_selection_pivot[df_selection_pivot['Kalenderwoche'] == week]
        if not sample_week.isin({'Symptom': [symptom]})['Symptom'].any():
            new_row = [symptom, week, 0]
            new_rows.append(new_row)
            
            
new_values = pd.DataFrame(new_rows, columns=['Symptom','Kalenderwoche','Anzahl'])

#append these illnesses with 0 'Anzahl'
df_selection_pivot= df_selection_pivot.append(new_values)
df_weekly_symptoms = df_selection_pivot.sort_values('Kalenderwoche').reset_index(drop=True)
df_weekly_symptoms = df_weekly_symptoms.sort_values(['Kalenderwoche','Symptom'],ascending=[True,True])





In [6]:
df_weekly_symptoms.to_pickle('df_weekly_symptoms.pkl')

In [7]:
#create a dictonary of dataframes per symptom
df_dict_weeks_symptomps = {}
symptoms = set(df_weekly_symptoms["Symptom"])
symptoms = list(symptoms)
symptoms.sort()
dataframes = [df_weekly_symptoms[df_weekly_symptoms["Symptom"]=='{}'.format(symptom)] for symptom in symptoms]
dict_symptomps = dict(zip(symptoms,dataframes))

In [105]:
#To filter symptoms. Copy symptoms contains the original name and short symp the shortend version

def filter_symptoms(x):
    match = re.search(r'([A-Z]+[^\s]*)\s',x)
    if match:
        return match.group(1)
    
to_be_changed_symptoms = []
for i, symp in enumerate(dict_symptomps):
    to_be_changed_symptoms.append(symp)


short_symp = map(filter_symptoms, to_be_changed_symptoms)
short_symp = list(short_symp)

copy_symptoms = symptoms.copy()

while None is short_symp:
    indx = short_symp.index(None)
    del short_symp[indx]
    del copy_symptoms[indx]
    



In [106]:
print(short_symp)

['Protein', None, 'Abschuppen', 'Absinken', None, None, 'Anzeichen', None, 'Anämie,', 'Appetit-', None, 'Atemstillstand,', None, 'Atrophie', 'Atypische', 'Aufnahme', 'Aufnahme', 'Ausschlag', 'Ausschlag', 'Ausschlag,', 'Ausschluss', None, None, 'Beläge,', None, 'Blutstuhl,', None, 'Blutung', None, 'Brennen,', None, None, None, 'Demenz,', None, 'Durchfall,', 'Durchfall,', None, 'EEG,', 'EEG,', None, 'Eiteransammlung', 'Entzündung', 'Entzündung', 'Entzündung', None, None, 'Erbrechen', 'Erregtheit', 'Erschöpfung,', 'Erythema', None, 'Fetales', None, 'Flächige', 'Fokus', None, None, None, None, None, None, None, None, None, 'Granulomatosis', None, 'Haut-', 'Haut-,', None, 'Hautausschlag,', 'Hauteinblutungen', None, 'Hautinfiltrationen,', None, 'Hautläsion,', 'Hautläsion,', 'Herabhängen', None, None, None, 'Herzfehler', None, None, 'Herzrhythmusstörung', 'Himbeer-', None, 'Hirndruckzeichen,', None, 'Hirnnervenstörung,', None, None, 'Husten', 'Hämatokrit', None, 'Impetigo/Pyodermie', 'Infekti

In [92]:
#The indices of the remaining symptoms I am interested in

indices = [4,11,12,13,14,16,18,20,21,24,25,26,27,28,36,37,38,39,41,42,46,47,48,49,52,53,67,70,71,76,77,111]
reduced_keys = [copy_symptoms[i] for i in indices]
print(short_symp)
dataframes_for_short = [df_weekly_symptoms[df_weekly_symptoms["Symptom"]=='{}'.format(symptom)] for key in reduced_keys]
dict_symptomps_shortened = dict(zip(reduced_keys,dataframes))



['Protein', 'Abschuppen', 'Absinken', None, 'Anzeichen', 'Anämie,', 'Appetit-', 'Atemstillstand,', 'Atrophie', 'Atypische', 'Aufnahme', 'Aufnahme', 'Ausschlag', 'Ausschlag', 'Ausschlag,', 'Ausschluss', None, 'Beläge,', 'Blutstuhl,', 'Blutung', 'Brennen,', None, 'Demenz,', 'Durchfall,', 'Durchfall,', 'EEG,', 'EEG,', 'Eiteransammlung', 'Entzündung', 'Entzündung', 'Entzündung', None, 'Erbrechen', 'Erregtheit', 'Erschöpfung,', 'Erythema', 'Fetales', 'Flächige', 'Fokus', None, None, None, None, 'Granulomatosis', 'Haut-', 'Haut-,', 'Hautausschlag,', 'Hauteinblutungen', 'Hautinfiltrationen,', 'Hautläsion,', 'Hautläsion,', 'Herabhängen', None, 'Herzfehler', None, 'Herzrhythmusstörung', 'Himbeer-', 'Hirndruckzeichen,', 'Hirnnervenstörung,', None, 'Husten', 'Hämatokrit', 'Impetigo/Pyodermie', 'Infektion', 'Infektion', None, 'Katarrh', 'Klinische', 'Klinisches', 'Klinisches', 'Klinisches', 'Klinisches', 'Klinisches', 'Klinisches', 'Klinisches', 'Klinisches', 'Klinisches', 'Klinisches', None, None

In [98]:
l = [1, None, 2, None]
# del l[1]
l.index(None)

1

In [8]:
rki_values = np.zeros((len(dict_symptomps),261))
google_values = np.zeros((len(dict_symptomps),261))
pytrend = TrendReq()

for i, symptom in enumerate(dict_symptomps):
        rki_timeseries = dict_symptomps['{}'.format(symptom)]
        rki_timeseries = rki_timeseries[-261:]
        rki_values_unnormlised = rki_timeseries['Anzahl'].values
        rki_values[i,:] = rki_values_unnormlised/max(rki_values_unnormlised)+0.0000001
        
        try:
            pytrend.build_payload(kw_list=['{}'.format(symptom)],geo="DE",timeframe='today 5-y',cat=0)
            google_trend_per_syndrom = pytrend.interest_over_time()
            google_values_unnormlised = google_trend_per_syndrom['{}'.format(symptom)].values
            print(google_values_unnormlised)
            google_values[i,:] = google_values_unnormlised/max(google_values_unnormlised)+0.0000001
        except:
            print('exception')


exception
[ 75  93  59  40  68  36  48  46  48 100  56  44  53  48  52  30  65  62
  41  49  64  50  57  19  31  38  38  30  43  52  52  42  37  52  35  23
  29  34  57  28  45  55  33  53  42  38  43  70  74  49  67  49  66  39
  46  86  71  34  23  73  48  43  52  55  33  37  32  32  39  39  73  31
  54  59  51  26  27  44  25  13  25  72  42  67  57  36  29  64  12  52
  23  74  46  46  57  23  28   7  27  26  75  69  49  78  73  60  71  50
  50  41  58  30  62  17  57  88  59  66  51  29  55  64  39  70  39  68
  52  41  48  29  12  29  17  52  50  89  73  38  50  49  49  65  60  48
  27  81  38  67  20  25  76  44  70  55  61  59  90  45  45  81  88  16
  49  58  52  73  61  29  64  54  54  56  47  45  26  26  28  38  22  40
  46  54  56  30  23  40  17  45  45  62  44  38  43  37  41  89  69  43
  73  36  33  12  35  44  65  69  44  78  64  20  58  58  45  30  52  56
  47  66  44  50  34  26  34  52  21  74  60  30  47  51  68  55  35  25
  25  26  45  20  24  39  47  24  67  48 

/home/student/a/aabbood/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


[ 43  34  42  44  46  33  22  45  43  35  61  39  32  52  31  32  35  42
  31  29  45  43  36  51  50  44  30  43  47  38  31  36  35  28  36  35
  46  33  37  33  33  43  21  35  58  41  76  47  37  43  33  35  41  41
  31  60  29  45  44  33  45  33  48  47  33  38  25  53  55  34  40  18
  45  29  60  27  40  30  39  51  52  45  43  51  40  52  53  26  38  24
  67  53  37  43  36  24  27  39  61  50  61  44  39  28  58  31  49  43
  30  19  27  41  30  53  30  31  37  43  35  45  45  43  54  26  28  33
  48  47  50  39  37  45  58  44  28  46  37  41  26  39  55  48  44  49
  50  48  40  42  44  51  56  56  46  46  32  50  47  52  46  51  49  44
  53  34  46  34  47  36  51  38  32  61  36  38  45  43  40  37  40  39
  55  41  39  45  46  42  45  35  48  40  39  35  35  41  25  42  33  32
  28  37  31  55  42  42  46  40  42  35  40  39  46  33  35  40  50  35
  37  45  47  39  31  37  32  21  39  25  22  48  37  40  28  30  53  37
  39  40  46  50  29  28  31  27  30  53  50  44  4

exception
[ 57  62  57  54  52  61  55  64  58  52  58  59  62  67  61  60  68  60
  64  65  79  61  70  78  70  66  66  65  63  66  65  64  68  66  50  72
  78  68  67  73  67  56  58  63  68  60  80  74  74  62  75  77  70  65
  68  75  70  62  69  68  67  70  67  65  70  59  66  72  68  79  73  70
  70  70  73  78  78  67  66  79  64  63  71  65  71  71  70  73  70  77
  76  76  65  69  76  67  79  82  74  88  72  77  77  80  75  81  78  73
  76  78  78  81  70  80  75  69  64  66  83  71  78  77  77  79  89  80
  75  70  80  76  81  92  73  72  69  82  64  67  69  84  84  69  85  73
  80  69  80  79  78  80  88  81  87  97  92  93  89  83  87  78  85  90
  90  85  79  89  80  86  89  85  81  84  77  81  74  81 100  79  87  87
  82  86  79  83  80  76  80  88  84  78  80  82  71  83  72  77  84  85
  87  80  84  82  87  81  80  84  82  82  87  93  80  88  87  78  78  81
  70  80  86  75  77  83  84  76  77  81  80  77  83  78  90  92  84  84
  83  87  74  76  73  80  80  70  75  81 

exception
[ 15  76  16  30  15  16  16  15  48  31  16  16  31  16  15  32  47   0
   0  16   0  33  17  17  33  51  33   0  16  30  46  16  64  15  46  46
  61  29  30  74  44  58  15  15  18  33  57  57  28  71  44  43  43   0
  30  46  43  59  61  74  79  16  60  14  44  32  33  17  17  17  48  16
   0  17  33  17  72 100  33  67  49  16  32  32  50  16  46  15  31  30
  61  30   0   0  15  61   0  17  14  42  28  14  28  15  44  14   0  44
  59  31  30   0  15  60  30  29  16  29  15  46  15  28  44  31  44  30
  30  93  32  16   0  46  30  45   0  58   0  29   0  14  14  14  29  42
  14  71  29  29  53  17  40  26  40   0  40  65  26   0  26  13  27   0
  29  14  14  41  13   0   0  42  14  40  14  26  27  41  29  28  59  60
   0  63  16   0  15  15  58  15  15   0  29  14   0  28  14  14  28  70
  69  27  44  16  13  51  26  26  26  26  13  52  51  39  13  13  14  40
  41  13  13   0  13  14  15   0  14  28   0  27   0  13  14  26   0  39
  13  27  39  13   0  26  50  25  25  38 

[ 48  36  50  36  46  55  42  48  39  45  34  43  35  42  35  49  32  43
  31  28  33  35  38  41  45  38  64  43  42  21  29  34  31  29  39  36
  27  39  34  31  38  36  48  39  46  39  43  50  36  49  41  49  47  40
  34  37  52  47  38  40  37  60  59  47  41  47  41  61  47  52  42  31
  34  47  47  38  52  44  33  39  25  40  42  41  30  47  41  39  43  43
  37  31  56  37  50  43  51  38  46  49  33  52  43  29  35  39  38  43
  48  42  50  32  36  40  37  43  42  43  42  43  37  42  45  39  37  34
  51  29  47  51  42  53  38  34  42  49  51  49  53  52  42  48  60  37
  36  58  56  55  51  55  52  46  51  46  45  39  52  52  37  64  67  47
  62  55  52  43  46  49  47  58  54  46  43  44  44  37  48  44  34  49
  53 100  43  42  51  46  49  47  32  60  34  35  38  43  33  40  36  33
  42  38  40  48  48  43  46  46  59  50  40  40  37  51  33  50  44  50
  37  49  48  48  47  42  45  41  56  53  49  48  43  43  45  42  45  48
  48  48  65  31  48  32  39  59  38  50  43  39  4

exception
[ 23  68  23  22  22  24  24  45  24  23  24   0   0  24  23   0  23  24
  23  48 100  25  25  25  49  25   0   0   0  23   0   0  48  46   0  46
   0  22   0   0  22  22  44  23   0  25   0  21  41  21  44   0  65  22
  23   0   0  22   0   0   0   0  23  22  22  24  25   0  26  25  24  24
  47   0   0  25   0  25   0  25  49  24  48  24   0  24   0   0   0  23
  23  23  45   0  45   0   0  26  21   0  21  42   0   0   0   0  22  44
   0  23  23   0  45   0  22  43  23  43  22   0  22   0  22  69  22  22
   0  69  47  70  23   0   0  23  43   0   0   0  22   0  22  64  64   0
  21  21  43   0  26   0   0  39   0   0   0  20  20  39  59  20   0  42
   0   0   0  20  20  22   0   0   0   0   0  59  21   0   0   0  22   0
  45   0   0   0  23  22   0  45  22  22  22  43  21  21  20  21  21   0
   0  61   0  23   0  19   0  19  19   0  58  39  19  38  20   0  21  20
  21   0  58   0  58  41  44   0  21   0   0   0  41   0  21  39   0  39
   0   0   0  19   0   0  37  19   0   0 

[ 70  56  53  47  44  45  40  37  35  38  41  38  29  33  30  27  28  25
  27  28  25  22  19  19  19  18  23  22  23  26  31  34  41  43  40  44
  39  37  38  39  37  40  38  37  48  43  37  34  35  43  46  45  51  50
  57  50  52  44  44  41  42  36  32  28  28  35  36  29  32  26  26  25
  24  28  26  26  22  23  24  25  29  30  28  34  41  40  48  47  47  48
  45  48  49  52  57  62  65  70  59  52  60  68  71  81  91  82  69  61
  59  51  49  47  44  42  42  37  35  35  39  38  34  27  28  30  28  26
  28  26  26  25  24  25  28  32  38  43  49  52  56  54  51  53  45  44
  45  45  51  51  69  67  54  50  50  64  66  69  71  72  72  73  77  75
  61  52  44  43  42  48  44  38  34  36  35  33  30  30  31  32  29  32
  28  26  30  30  29  32  32  33  43  46  51  52  51  53  55  59  63  60
  64  68  72  85  72  68  66  76  89  82  80  72  60  64  55  52  51  49
  48  45  43  43  42  45  43  37  31  32  31  27  28  32  32  30  30  29
  30  30  32  35  42  51  51  56  56  58  56  55  5

exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 100
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   

In [72]:
#pytrend = TrendReq()


test = 'Husten'
pytrend.build_payload(kw_list=['Husten'],geo="DE",timeframe='today 5-y',cat=0)
google_trend_per_syndrom = pytrend.interest_over_time()
google_values_unnormlised = google_trend_per_syndrom['Husten'].values
google_values= google_values_unnormlised/max(google_values_unnormlised)+0.0000001
print(google_values)


In [226]:
correlation_values = np.zeros((len(dict_symptomps)))
for i in range(len(dict_symptomps)):
    try:
        output_tuple = sci.pearsonr(google_values[i],rki_values[i])
        correlation_values[i] = output_tuple[0]
    except RuntimeWarning:
        print('shits on fire')

/Users/Auss/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:3021: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
